In [ ]:
import os

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

import SSLimPy.interface.sslimpy as sslimpy
import SSLimPy.LIMsurvey.Covariance as sco
from SSLimPy.LIMsurvey import PowerSpectra
from SSLimPy.LIMsurvey import higherorder as sho
from SSLimPy.utils import utils as utl

In [ ]:
envkey = "OMP_NUM_THREADS"
# Set this environment variable to the number of available cores in your machine,
# to get a fast execution of the Einstein Boltzmann Solver
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))
os.environ[envkey] = str(12)
print("The value of {:s} is: ".format(envkey), os.environ.get(envkey))

In [ ]:
cosmodict = {"h": 0.67, "Omegam": 0.32, "Omegab": 0.04, "As": 2.1e-9, "mnu": 0.06}
settings = {
    "code": "class",
    "do_RSD": True,
    "nonlinearRSD": True,
    "FoG_damp": "ISTF_like",
    "halo_model_PS": True,
    "Smooth_window": True,
    "kmin": 0.004 * u.Mpc**-1,
    "kmax": 0.1 * u.Mpc**-1,
    "nk": 50,
}

# Deactivate LIM for testing
astrodict = {
    "model_type":"ML",
    "model_name":"MassPow",
    "model_par": {
        "A":1,
        "b":0
    }
}
Asslimpy = sslimpy.sslimpy(settings_dict=settings, cosmopars=cosmodict, astropars=astrodict)

pobs = PowerSpectra.PowerSpectra(Asslimpy.fiducialcosmo, Asslimpy.fiducialastro)

In [ ]:
k = pobs.k
pk = pobs.fiducialcosmo.matpow(k, 0)
I2 = pobs.higher_halomoments([1, 2], k, k, moment=2, bias_order=0, kbias=0)
print(I2.shape)
I1 = pobs.higher_halomoments([1, 2], k, moment=1, bias_order=0, kbias=0)
print(I1.shape)

In [ ]:
kinterp = np.array([0,1e-3,1e-2,0.1,1])
logki = np.log(kinterp)
logkg = np.log(k.value)
logpg = np.log(pk.value)
logpi = utl.linear_interpolate(logkg,logpg,logki)
plt.plot(logkg,logpg)
plt.scatter(logki, logpi)
plt.show()

In [ ]:
def get_vectors(k1,mu1,ph1,k4,mu4,ph4):

    k14, mu14, ph14 = utl.addVectors(k1, mu1, ph1, k4, mu4, ph4)
    return sho.vF2(k14, mu14, ph14, k1, -mu1, ph1 + np.pi),sho.vF2(k14, mu14, ph14, k4, -mu4, ph4 + np.pi)

In [ ]:
def fake_add_vecotrs(k1,mu1,ph1,k2,mu2,ph2):
    ra1 = (1-mu1**2)
    if ra1<0:
        ra1=0
    s1 = np.sqrt(ra1)

    ra2 = (1-mu2**2)
    if ra2<0:
        ra2=0
    s2 = np.sqrt(ra2)


    x1, y1, z1 = k1 *  s1 * np.cos(ph1), k1 *  s1 * np.sin(ph1), k1 * mu1
    x2, y2, z2 = k2 *  s2 * np.cos(ph2), k2 *  s2 * np.sin(ph2), k2 * mu2
    x,y,z = x1+x2, y1+y2, z1+z2
    k = np.sqrt(x**2+y**2+z**2)
    mu = z/k
    phi = np.arctan2(y,x)
    return k,mu,phi

In [ ]:
k = np.geomspace(2e-3, 2, 50) * u.Mpc**-1
Pk = pobs.fiducialcosmo.matpow(k, 0)

k1, mu1, ph1 = 0.1, 0.4, 0.2
k2, mu2, ph2 = 0.1, -0.4, 0.2 + np.pi
k3, mu3, ph3 = 0.08, 0.5, 0
k4, mu4, ph4 = 0.08, -0.5, np.pi

In [ ]:
mui = -1
mu2l = np.linspace(-1,0.9,50)
f = []
for mui2 in mu2l:
    a, _ = get_vectors(k1,mui,ph1,k4,-mui2,ph4)
    f.append(a)

plt.plot(mu2l, f)

In [ ]:
mui = -1
mu2l = [-1,-0.999,-0.9]

In [ ]:
for mui2 in mu2l:
    print(utl.addVectors(k1,mui,ph1,k3,mui2,ph2))
    print(fake_add_vecotrs(k1,mui,ph1,k3,mui2,ph2))

In [ ]:
B = np.empty_like(k.value)
T = np.empty_like(k.value)

for ik, ki in enumerate(k):
    k23, mu23, ph23 = utl.addVectors(ki.value, mu2, ph2, k3, mu3, ph3)
    Bi = sho.BispectrumLO(
        ki.value, mu1, ph1, k23, mu23, ph23, k4, mu4, ph4, k.value, Pk.value
    )
    B[ik] = Bi
    Ti = sho.TrispectrumL0(ki.value, mu1, ph1, ki.value, mu2, ph2,
                           k3, mu3, ph3, k4, mu4, ph4,
                           k.value, Pk.value,
    )
    T[ik] = Ti

In [ ]:
plt.loglog(k, B/np.max(B), label="Bi-Spectrum/max", c= "blue")
plt.loglog(k, T/np.max(T), label="Tri-Spectrum/max")
plt.loglog(k,Pk/np.max(Pk), label="PowerSpectrum/max")
plt.xlabel(r"$k [\mathrm{Mpc}^{-1}]$")
plt.legend()

In [ ]:
k = np.geomspace(2e-3, 20, 20) * u.Mpc**-1

kpivots = np.geomspace(2e-3,20,20)* u.Mpc**-1


I1 = np.array([pobs.higher_halomoments(1, k, moment=1, bias_order=0, kbias=0) for kpivot in kpivots])
I2 = np.array([pobs.higher_halomoments(1, k, kpivot, moment=2, bias_order=0, kbias=0) for kpivot in kpivots])
I3 = np.array([pobs.higher_halomoments(1, k, kpivot, kpivot, moment=3, bias_order=0, kbias=0) for kpivot in kpivots])
I4 = np.array([pobs.higher_halomoments(1, k, kpivot, kpivot, kpivot, moment=4, bias_order=0, kbias=0) for kpivot in kpivots])

In [ ]:
alpha = np.linspace(0.5,1,20)

for ia, ai in enumerate(alpha):

    plt.loglog(k,I1[ia,:]**4, alpha=ai, c="red")
    plt.loglog(k,I2[ia,:] * I1[ia,:]**2, alpha=ai, c="blue")
    plt.loglog(k,I3[ia,:] * I1[ia,:], alpha=ai, c="green")
    plt.loglog(k,I4[ia,:], alpha=ai, c="orange")

In [ ]:
k = pobs.k
pk = pobs.cosmology.matpow(k,0)

k1, mu1, ph1 = 0.1, 0.0, 0.2
k2, mu2, ph2 = 0.1, 0.0, 0.2 + np.pi
k3, mu3, ph3 = 0.08, 0.5, 0
k4, mu4, ph4 = 0.08, -0.5, np.pi

k13, mu13, ph13 = utl.addVectors(k1, mu1, ph1, k3, mu3, ph3)

print(sho.BispectrumLO(k2, mu2, ph2, k13, mu13, ph13, k4, mu4, ph4, k.value, pk.value))
print(sho.BispectrumLO(k2, mu2, ph2, k4, mu4, ph4, k13, mu13, ph13, k.value, pk.value))

In [ ]:
pobs.compute_power_spectra()
pobs.compute_power_spectra_moments()
ng_Corr = sco.nonGuassianCov(pobs)

In [ ]:
gauss = sco.Covariance(pobs)

gg = gauss.gaussian_cov()

In [ ]:
CT0 = ng_Corr.integrate_4h()

In [ ]:
CB0 = ng_Corr.integrate_3h()

In [ ]:
CP0 = ng_Corr.integrate_2h()

In [ ]:
CV0 = ng_Corr.integrate_1h()

In [ ]:
plt.loglog(k, CT0[:, 25, 0, 0])
plt.loglog(k, CB0[:, 25, 0, 0])
plt.loglog(k, CP0[:, 25, 0, 0])
plt.loglog(k, CV0[:, 25])
plt.scatter(k[25], gg[25, 0, 0, 0])

In [ ]:
import itertools
from scipy.special import roots_legendre

In [ ]:
k = np.geomspace(2, 20, 20) * u.Mpc**-1
kl = len(k)

wl = 7
xi, w = roots_legendre(wl)
mu = xi
z=1

indexmenge = range(wl)
I1 = np.empty((kl, wl))
for imu1 in indexmenge:
    Ii = pobs.halo_temperature_moments(z, k, mu[imu1], bias_order=1, moment=1)
    I1[:,imu1] = Ii.value

indexmenge = itertools.product(range(wl), repeat=2)
I2 = np.empty((kl, kl, wl, wl))
for imu1, imu2, in indexmenge:
    Iij = pobs.halo_temperature_moments(z, k, k, mu[imu1], mu[imu2], bias_order=1, moment=2)
    I2[:, :, imu1, imu2] = Iij.value

indexmenge = itertools.product(range(wl), repeat=2)
indexmengek = itertools.product(range(kl), repeat=2)
I3 = np.zeros((kl, kl, wl, wl))
for imu1, imu2, in indexmenge:
    for ik1, ik2 in indexmengek:
        Iijk = pobs.halo_temperature_moments(z, k[ik1], k[ik2], k[ik1], mu[imu1], mu[imu2], -mu[imu1], bias_order=1, moment=3)
        I3[ik1, ik2, imu1, imu2] = Iijk.value

In [ ]:
print(np.any(np.isnan(I1)), I1.shape)
print(np.any(np.isnan(I2)), I2.shape)
print(np.any(np.isnan(I3)), I3.shape)

In [ ]:
plt.loglog(k,I1[:,0])
plt.loglog(k,I2[:,0,0,0])
plt.loglog(k,I3[:,0,0,0])

In [ ]:
I3

In [ ]:
f = pobs.ft_NFW(k, [5e8, 8e8, 11e8] * u.Msun, 0)
plt.loglog(k, f)

In [ ]:
mu1 = np.linspace(-1, 1, 200)
mu2 = np.linspace(-1, 1, 7)
mu2 = np.linspace(-0.99, 0.99, 7)

Tmu = np.empty_like(mu1)

for mui2 in mu2:
    for imu, mui in enumerate(mu1):
        Tmu[imu] = sho.TrispectrumL0(
            k1,
            mui,
            0.1,
            k1,
            -mui,
            np.pi + 0.1,
            k3,
            mui2,
            1,
            k3,
            -mui2,
            np.pi + 1,
            k,
            pk,
        )
        if np.isnan(Tmu[imu]):
            print(mui, mui2)
            sho.TrispectrumL0(
                k2, mu2, ph2, k3, mu3, ph3, k4, mu4, ph4, k1, mu1, ph1, k, pk
            )
    plt.plot(mu1, Tmu, label=r"$\mu_2={:.2f}$".format(mui2))

plt.xlabel(r"$k_1$ cosine angle $\mu_1$")
plt.ylabel("Trispectrum squeezed $T_0$")
legend = plt.legend(title=r"$k_2$ cosine angle", fancybox=False, edgecolor="black")
legend.get_frame().set_facecolor((1, 1, 1, 0))  # Fully transparent background
legend.get_frame().set_edgecolor("black")  # Solid black border
legend.get_frame().set_linewidth(2)  # Set border width

# Set the plot border (spines) and tick width
ax = plt.gca()  # Get the current axes3505071717386430.5
for spine in ax.spines.values():  # Set the width of all spines (borders)
    spine.set_linewidth(2)

# Set tick parameters
ax.tick_params(width=2, size=4, direction="in")  # Set the tick width to 2
# Show plot
plt.show()